In [23]:
#Clone Repository and Move into It

%cd /content
!rm -rf deepvision-unlearning
!git clone https://github.com/sri-sruthi/deepvision-unlearning.git
%cd deepvision-unlearning

/content
Cloning into 'deepvision-unlearning'...
remote: Enumerating objects: 238, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 238 (delta 0), reused 0 (delta 0), pack-reused 234 (from 1)
Receiving objects: 100% (238/238), 50.85 KiB | 801.00 KiB/s, done.
Resolving deltas: 100% (118/118), done.
/content/deepvision-unlearning


In [24]:
pwd

'/content/deepvision-unlearning'

In [25]:
#Install Dependencies

!pip install -r requirements.txt -q

In [26]:
#Check GPU

!nvidia-smi

Sat Nov 29 10:29:00 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   75C    P0             34W /   70W |     306MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [27]:
#Printing the current working directory

import os
os.getcwd()

'/content/deepvision-unlearning'

In [28]:
#Add the repository to the Python path

import sys
sys.path.append('/content/deepvision-unlearning')

In [29]:
#CIFAR Loader Test

from data.cifar_loaders import get_cifar10_loaders

train_loader, test_loader = get_cifar10_loaders()
images, labels = next(iter(train_loader))
print("Images batch shape:", images.shape)
print("Labels batch shape:", labels.shape)

100%|██████████| 170M/170M [00:04<00:00, 42.3MB/s]


Images batch shape: torch.Size([128, 3, 32, 32])
Labels batch shape: torch.Size([128])


In [30]:
#Model Forward Pass Test

import torch
from models.resnet18 import get_resnet18

device = "cuda" if torch.cuda.is_available() else "cpu"
model = get_resnet18(num_classes=10).to(device)

outputs = model(images.to(device))
print("Output shape:", outputs.shape)

Output shape: torch.Size([128, 10])


In [31]:
#Set PYTHONPATH

import sys
sys.path.append('/content/deepvision-unlearning')

In [32]:
#Mini Training Debug Run on ResNet18 on CIFAR-10 for 5 epochs

!PYTHONPATH=/content/deepvision-unlearning python experiments/train.py --model resnet18 --dataset cifar10 --epochs 5 --batch_size 128


Epoch 1/5
Train Acc: 43.28% | Val Acc: 52.99%
Saved Best Model

Epoch 2/5
Train Acc: 57.40% | Val Acc: 61.48%
Saved Best Model

Epoch 3/5
Train Acc: 63.54% | Val Acc: 63.21%
Saved Best Model

Epoch 4/5
Train Acc: 67.51% | Val Acc: 68.66%
Saved Best Model

Epoch 5/5
Train Acc: 70.33% | Val Acc: 71.09%
Saved Best Model
Training complete! Best Acc: 71.09


In [33]:
#To verify where the checkpoint was stored from the 5-epoch debug run

!ls -R ./models

!find /content -name "*.pth"

./models:
__init__.py	mobilevit.py  resnet18_cifar10_best.pth  vit_tiny.py
mobilenetv2.py	__pycache__   resnet18.py

./models/__pycache__:
__init__.cpython-312.pyc     resnet18.cpython-312.pyc
mobilenetv2.cpython-312.pyc  vit_tiny.cpython-312.pyc
mobilevit.cpython-312.pyc
/content/deepvision-unlearning/models/resnet18_cifar10_best.pth


In [34]:
#To View Help Text

!PYTHONPATH=/content/deepvision-unlearning python experiments/run_unlearning.py -h

usage: run_unlearning.py [-h] [--method {ga,sisa}] [--model MODEL]
                         [--dataset DATASET] [--forget-type {class,sample}]
                         [--forget-class FORGET_CLASS]
                         [--forget-count FORGET_COUNT]
                         [--batch_size BATCH_SIZE] [--save_dir SAVE_DIR]

options:
  -h, --help            show this help message and exit
  --method {ga,sisa}
  --model MODEL
  --dataset DATASET
  --forget-type {class,sample}
  --forget-class FORGET_CLASS
  --forget-count FORGET_COUNT
  --batch_size BATCH_SIZE
  --save_dir SAVE_DIR


In [35]:
#Gradient Ascent Unlearning Dry Run

!PYTHONPATH=/content/deepvision-unlearning python experiments/run_unlearning.py \
--method ga --model resnet18 --dataset cifar10 \
--forget-type sample --forget-count 100 \
--batch_size 64 --save_dir ./models/

Unlearning via Gradient Ascent: 100% 50/50 [00:02<00:00, 17.61it/s]
Saved unlearned model: resnet18_cifar10_ga_sample.pth


In [36]:
#Testing the evaluation engine

!PYTHONPATH=/content/deepvision-unlearning python experiments/run_evaluation.py \
--model-path ./models/resnet18_cifar10_ga_sample.pth \
--model resnet18 --dataset cifar10

Forgetting Effectiveness: 0.3911
Retention Accuracy: 39.37
